In [ ]:
import pandas as pd

df_train = pd.read_csv("training_task_a_tweets.csv")
df_test = pd.read_csv("test_a.csv")


In [ ]:
df_train.head()

,id,in_reply_to_status_id_str,text,user.verified,user.followers_count,retweet_count,favorite_count,entities.hashtags,entities.urls,label
0,552785374507175936,552783667052167168,MT @euronews France: 10 dead after shooting at...,False,1828,1,2,"[{'indices': [70, 83], 'text': 'CharlieHebdo'}]",[],comment
1,552786226546495488,552785374507175936,@j0nathandavis They who? Stupid and partial op...,False,405,0,0,[],[],deny
2,552789345628069888,552786226546495488,"@nanoSpawn Socialists, Antisemites, anti zioni...",False,1828,0,0,[],[],comment
3,552792862610694144,552783667052167168,@euronews @TradeDesk_Steve A French crime of p...,False,26,0,0,[],[],query
4,552943855021330432,552783667052167168,"@euronews LOL. 5 million Muslims in France, wh...",False,14,0,0,[],[],comment


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn import preprocessing

In [ ]:
import nltk
import re
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
df_train.label.unique()

array(['comment', 'deny', 'query', 'support'], dtype=object)

In [ ]:
df_train["text"]=df_train["text"].str.lower()
df_test["text"]=df_test["text"].str.lower()

In [ ]:
def getLemmText(text):
 tokens=word_tokenize(text)
 lemmatizer = WordNetLemmatizer()
 tokens=[lemmatizer.lemmatize(word) for word in tokens]
 return ' '.join(tokens)
def stop_words(text):
  text = REPLACE_BY_SPACE_RE.sub(' ', text)
  text = BAD_SYMBOLS_RE.sub('', text)
  text = ' '.join(word for word in text.split() if word not in STOPWORDS)
  return text
def getStemmText(text):
 tokens=word_tokenize(text)
 ps = PorterStemmer()
 tokens=[ps.stem(word) for word in tokens]
 return ' '.join(tokens)

In [ ]:
df_train['text'] = list(map(stop_words,df_train['text']))
df_train['text'] = list(map(getLemmText,df_train['text']))
df_train['text'] = list(map(getStemmText,df_train['text']))

df_test['text'] = list(map(stop_words,df_test['text']))
df_test['text'] = list(map(getLemmText,df_test['text']))
df_test['text'] = list(map(getStemmText,df_test['text']))

In [ ]:
label_encoder = preprocessing.LabelEncoder()
df_train['label'] = label_encoder.fit_transform(df_train.label.values)
df_test['label'] = label_encoder.fit_transform(df_test.label.values)

In [ ]:
print(df_train['label'].value_counts())

0    2896
3     633
2     358
1     335
Name: label, dtype: int64


In [ ]:
df_train.drop(df_train.columns[[0, 1,3,4,5,6,7,8]], axis = 1, inplace = True)
df_test.drop(df_test.columns[[0, 1,3,4,5,6,7,8]], axis = 1, inplace = True)

In [ ]:
df_test.head()

,text,label
0,todayshow big expens payload kill 36 peopl,0
1,todayshow mani isi kill,2
2,todayshow halliejackson mother give beak men m...,0
3,todayshow look like there noth wipe whole land...,0
4,todayshow next one could daddi bomb destruct m...,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
stopset = set(stopwords.words('english'))
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
vectorizer = TfidfVectorizer(use_idf=True, lowercase=True, strip_accents='ascii', stop_words=stopset)

y_train = df_train["label"]
y_test = df_test["label"]
x_train = vectorizer.fit_transform(df_train["text"])
x_test = vectorizer.transform(df_test["text"])




In [ ]:
#Using Naive Bayes for classification

classifier = MultinomialNB().fit(x_train, y_train)


In [ ]:
y_pred = classifier.predict(x_test)

In [ ]:
import sklearn.metrics as metrics 

In [ ]:
print(metrics.confusion_matrix(y_test,y_pred),"\n")
print(metrics.accuracy_score(y_test,y_pred))

[[771   0   0   0]
 [ 92   0   0   0]
 [ 56   0   0   0]
 [ 91   0   0   0]] 

0.7633663366336634


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

In [ ]:
print(precision_score(y_test, y_pred , average="macro"))
print(recall_score(y_test, y_pred , average="macro"))
print(f1_score(y_test, y_pred , average="macro"))

0.19084158415841584
0.25
0.21645143177989892


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# using support vector machines for classification
from sklearn import svm
from sklearn.metrics import classification_report

classifier_linear = svm.SVC(kernel='linear')
classifier_linear.fit(x_train, y_train)
prediction_linear = classifier_linear.predict(x_test)

In [ ]:
print(metrics.confusion_matrix(y_test,prediction_linear),"\n")
print(metrics.accuracy_score(y_test,prediction_linear))

print(precision_score(y_test, prediction_linear , average="macro"))
print(recall_score(y_test, prediction_linear , average="macro"))
print(f1_score(y_test, prediction_linear , average="macro"))

[[734   0   0  37]
 [ 85   0   0   7]
 [ 54   0   0   2]
 [ 78   0   0  13]] 

0.7396039603960396
0.24803953020014613
0.2737168797480081
0.2564576074332172


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
